# Logistic regression via gradient descent

Instructions:

There are 6 problems (look for Problem 1, Problem 2, etc.). For each, replace the comment YOUR CODE HERE with your code. Do not modify the code in other ways. 

You will probably want to write test code while you're getting your code working. Mark your test code clearly and then remove it before submitting. 

Before submitting your code, restart the kernel and then run all cells.

v1.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import sys

The random seed is set so results are repeatable.

In [ ]:
np.random.seed(0)

## Gradients

The code used here to compute the gradient of a function is different from the code used in the linear regression assignment.  This code is more efficient.  Please read the code carefully.

In [ ]:
def partials_at_x(f, n, x):
    h = 0.000001
    y = f(x)
    partial_derivs = np.zeros(n)
    for i in range(n):
        x[i] += h
        yh = f(x)
        x[i] -= h
        partial_derivs[i] = yh - y
    partial_derivs = partial_derivs/h
    return partial_derivs
    
def gradient(f, n):
    return lambda x: partials_at_x(f, n, x)

In [ ]:
# test the gradient() function

def f(x):
    return 2*x[0]**2 + x[0]*x[1]

# print the gradient of f
f_grad = gradient(f, 2)
xs = [ np.array(x) for x in [[1.0, 2.0], [0.0, 0.0], [2.0, -1.0]] ]
for x in xs:
    print('x: {}, f_grad(x): {}'.format(x, f_grad(x)))

## Gradient descent

In gradient descent we want to find the value of x that minimizes (or maximizes) a function f.  We do this by starting with some x, computing the value of the gradient of f at x, and then using that value to make an adjustment to x.

In [ ]:
def grad_descent(f, n, alpha=0.1, n_iterations=1000, threshold=0.001):
  """ Return the value x for which function f(x) is minimum, and also
  the values of x along the path of gradient descent.  
  The algorithm terminates when iteration limit is reach or when the
  L1 norm of the change to x is below the threshold.  """
  
  f_grad = gradient(f, n)

  x = np.random.rand(n)           # note random starting point
  xs = [x.copy()]
  for i in range(n_iterations):
    delta = alpha * f_grad(x)
    x -= delta
    xs.append(x.copy())           # the copy is needed, else all values in xs will be the same
    if np.sum(np.absolute(delta)) < threshold:     # L1 norm
      return x, xs

  print('warning: reached iteration limit')
  return x, xs

In [ ]:
# test the grad_descent() function

# test on some 1D functions
f1 = lambda x: (x[0] - 1)**2
f2 = lambda x: 0.2*(x[0]**2) + np.sin(x[0])
for f in [f1, f2]:
  x, xs = grad_descent(f, 1)
  print('x = {}, f(x) = {:0.3f}'.format(x, f(x)))
    
# test on 2D function
f = lambda x: (x[0]-1)**2 + (x[1]+1)**2 + 2.0
x, xs = grad_descent(f, 2)
print('x = {}, f(x) = {:0.3f}'.format(x, f(x)))

Plot the value of the loss function as gradient descent proceeds.

In [ ]:
def plot_descent(f, xs):
  """ plot value of function f using history xs of gradient descent """
  plt.plot([f(x) for x in xs])
  plt.xlabel('iteration')
  plt.ylabel('f(x)')
  plt.title('Progress of gradient descent')
  plt.grid();

In [ ]:
# plot progress of gradient descent for function f
x, xs = grad_descent(f, 2)
plot_descent(f, xs)

## Binary Logistic regression

The key idea for training is that we want to use gradient descent to find the model parameters that minimize the loss function.  For binary classification with logistic regression, the loss function is "log loss".

#### Heart disease data set

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/grbruns/cst383/master/heart.csv")
df.info()

Prepare the training data set.

In [ ]:
df['output'] -= 1         # 0 = ok, 1 = heart disease

predictors = ['age', 'restbp', 'maxhr']
target = 'output'
X = df[predictors].values
y = df[target].values

# scale the data
X = StandardScaler().fit_transform(X)

# augment the data
X1 = np.c_[np.ones(X.shape[0]), X]        # add new first column of 1's

print(X1.shape)
print(y.shape)

In [ ]:
# plot output by rest BP
plt.scatter(X[:,1], y, );
plt.title('training data set')
plt.xlabel('resting blood pressure (scaled)')
plt.ylabel('output');

### Log loss

Note that to compute the loss we need both the model parameters and the training data.

#### Problem 1

In [ ]:
# implement sigmoid yourself, not with a library function
def sigmoid(x):
    return # YOUR CODE HERE

In [ ]:
# test sigmoid()

print(sigmoid(-1.0))
print(sigmoid(0))
print(sigmoid(0.8))

#### Problem 2

In [ ]:
# implement log loss yourself, not with a library function
def log_loss(b, X1, y):
    # YOUR CODE HERE
    # hints: 
    # - first make predictions, then compute loss
    # - be sure to end with a return statement

In [ ]:
# test log_loss() by computing the loss for various
# values of linear parameters b
m = 4
bs = np.array([[-0.2, 0.1, 0.3, -1.0],
              [0.8, 0.5, 0.6, 0.9],
              [0.05, 0.08, 0.02, 0.8]])
for i in range(bs.shape[0]):
    b = bs[i]
    print('b: {}, log_loss(b): {:0.3f}'.format(b, log_loss(b, X1, y)))

### Gradient descent with log loss

Now we can put together our gradient descent function and log_loss function to perform logistic regression.

#### Problem 3

In [ ]:
# hints:
# for gradient descent, the loss function will be log_loss() with the 
# training data X1, y "hardwired" in

loss = lambda b: # YOUR CODE HERE
b_estimated, history = grad_descent(# YOUR CODE HERE)
print(b_estimated)

Compute training loss and training accuracy.  Use a threshold of 0.5 when computing accuracy.

In [ ]:
pred = sigmoid(X1.dot(b_estimated))    
training_loss = loss(b_estimated)
training_acc = (y == (pred > 0.5)).mean()

print('training loss: {:0.3f}, training accuracy: {:0.3f}'.format(training_loss, training_acc))

This plot shows how log loss decreases during gradient descent.

In [ ]:
plot_descent(loss, history)

### Compare to the result from Scikit-Learn

In [ ]:
clf = LogisticRegression()
clf.fit(X, y)

print('Model coefficients:')
print(clf.intercept_)
print(clf.coef_)

print('\nScikit-Learn training accuracy: {:0.3f}'.format(clf.score(X,y)))     # training accuracy

## Multi-class logistic regression

If we have a multi-class classification problem, then we need to use a separate linear model for each output class.  If we have 3 output class, for example, from each input we will get a vector of three output values.  In multi-class logistic regression, we transform this vector to a vector of probabilities using the softmax function.

#### Iris data set

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/grbruns/cst383/master/iris.csv")
df.info()

In [ ]:
df['species'].value_counts()

Preprocess the data

In [ ]:
predictors = df.columns[:4]
X = df[predictors].values
y_raw = df['species'].values

# one-hot encoding of the target
df = pd.get_dummies(df)
target = df.columns[4:]
y = df[target].values

# scale the data
X = StandardScaler().fit_transform(X)

# augment the data
X1 = np.c_[np.ones(X.shape[0]), X]        # add new first column of 1's

print(X1.shape)
print(y.shape)
print(y_raw.shape)

### Softmax

#### Problem 4

In [ ]:
#### Implement softmax yourself
def softmax(x):
    # YOUR CODE HERE
    # don't forget the return statement at the end

Test softmax

In [ ]:
print(softmax(np.array([1.7, 2.4, 0.5])))
print(softmax(np.array([-0.2, 1.2, -0.1])))

#### Making predictions

We have a separate linear model for each output class.  If we have k output classes, and n predictors, then we need n+1 parameters for each class.  Instead of keeping the parameters for each class separate, we can put them in a single matrix B.  Each row of B will contain the parameters for one class.  In other words, B will have k rows and n+1 columns.

Think about the matrix multiplication that is needed here.  Previously we got a single output value for each training example by computing X1.dot(b).  How shall we combine X1 and B to get an array of outputs, one for each class?

Make predictions using a random set of parameters of the linear model.  Each row in array pred is a prediction.

In [ ]:
B = np.random.rand(X1.shape[1], y.shape[1])
pred = X1.dot(B)
pred[:5]

Apply softmax to each row, so that the values in each row sum to 1.

In [ ]:
pred = np.apply_along_axis(softmax, 1, pred)
print(pred[:5])

The generalization of log loss to more than 2 classes is cross entropy.

### Cross-entropy

#### Problem 5

In [ ]:
def cross_entropy(pred, y):
    """ cross-entropy of arrays pred and y, where y is one-hot encoded """
    return # YOUR CODE HERE

def cross_entropy_loss(b, X1, y):
    B = b.reshape((X1.shape[1], y.shape[1]))
    # YOUR CODE HERE
    # hints: 
    # - note that the parameters are passed in as a 1D array, but
    #   then reshaped
    # - as with log loss, first compute the predictions
    #   (see code above that shows how to make predictions)
    # - don't forget the return statement

Test cross_entropy().

In [ ]:
print(cross_entropy(np.array([0.98, 0.01, 0.01]), np.array([1, 0, 0])))
print(cross_entropy(np.array([0.90, 0.05, 0.05]), np.array([1, 0, 0])))
print(cross_entropy(np.array([0.40, 0.20, 0.20]), np.array([1, 0, 0])))
print(cross_entropy(np.array([0.40, 0.20, 0.20]), np.array([0, 1, 0])))

# class examples
print(cross_entropy(np.array([0.3, 0.61, 0.09]), np.array([0, 1, 0])))
print(cross_entropy(np.array([0.3, 0.61, 0.09]), np.array([1, 0, 0])))

In [ ]:
# checking an example from lecture
print(cross_entropy(np.array([[0.1, 0.5, 0.4],
                              [0.1, 0.1, 0.8],
                              [0.2, 0.6, 0.2]]),
                    np.array([[1, 0, 0],
                              [0, 0, 1],
                              [0, 1, 0]])))

Test cross_entropy_loss().

In [ ]:
# the size of b equals the number of columns in X1
# times the number of target classes
n = X1.shape[1] * y.shape[1]

b = np.linspace(0, 1, n)
print(cross_entropy_loss(b, X1, y))

b = np.linspace(-2, 2, n)
print(cross_entropy_loss(b, X1, y))

### Gradient descent with cross-entropy.

Using gradient descent, softmax, and cross entropy loss we can perform multi-class logistic regression.

#### Problem 6

In [ ]:
loss = # YOUR CODE HERE
num_parameters = # YOUR CODE HERE
b_estimated, history = grad_descent(loss, num_parameters, alpha=0.5, n_iterations=400, threshold=0.00001)
print(b_estimated)

In [ ]:
print(cross_entropy_loss(b_estimated, X1, y))

In [ ]:
plot_descent(loss, history)

Compute training set accuracy

In [ ]:
B1 = b_estimated.reshape((X1.shape[1], y.shape[1]))
pred = np.apply_along_axis(np.argmax, 1, X1.dot(B1))  
y1d  = np.apply_along_axis(np.argmax, 1, y)

# training accuracy
print('gradient descent training accuracy: {:0.3f}'.format((pred == y1d).mean()))

Compare results to Scikit Learn

In [ ]:
clf = LogisticRegression()
clf.fit(X, y_raw)

print('Scikit-Learn training accuracy: {:0.3f}'.format(clf.score(X,y_raw)))